In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import sqlite3
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve , auc
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv('more_preprocess.csv')

In [29]:
test = pd.read_csv('test_preprocess.csv')

In [4]:
fill_data = df.sample(n=100000, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
vectorizer = CountVectorizer(ngram_range=(1, 3),max_features=40000)
fill_data['destr_review'] = fill_data['destr_review'].fillna('')
test['destr_review'] = test['destr_review'].fillna('')
X_vec = vectorizer.fit_transform(fill_data['destr_review'])
test_vec = vectorizer.transform(test['destr_review'])

In [7]:
labels = fill_data['rating']

columns_not_to_drop = 'destr_review'
for col in fill_data.columns:
    if col not in columns_not_to_drop:
        fill_data.drop(columns=col, inplace=True)

columns_not_to_drop = 'destr_review'
for col in test.columns:
    if col not in columns_not_to_drop:
        test.drop(columns=col, inplace=True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_vec, labels, test_size=0.2, random_state=42)

In [22]:
rf_classifier = RandomForestClassifier()

In [23]:
param_grid = {
    'n_estimators': [20, 30, 40],  # Number of trees in the forest
    'max_depth': [None, 10, 11, 12],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

In [24]:
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 11, 12],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [20, 30, 40]},
             verbose=2)

In [25]:
grid_search.best_params_

{'max_depth': None,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 40}

In [26]:
# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 40}
Best Score: 0.4613625


In [27]:
best_rf = grid_search.best_estimator_
test_accuracy = best_rf.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.4632


In [30]:
test.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,review,destr_review
0,e592483866ae722803c6e40572f9ea4e,21525330,606d3b7ba5cc90e4069b1e225b84deea,Very cute and sweet story about a girl who tra...,Thu Jan 08 14:52:16 -0800 2015,Sun Jan 11 19:53:03 -0800 2015,Sun Jan 11 00:00:00 -0800 2015,Fri Jan 09 00:00:00 -0800 2015,5,0,"['cute', 'sweet', 'story', 'girl', 'travel', '...",cute sweet story girl travel neverland fall ca...
1,52e14841f8745f9761cb85d4ef60ef25,22848621,70388d316638176b827ea060839971ef,No plot regurgitation. \n This couple are what...,Sat Jul 04 20:37:28 -0700 2015,Fri Feb 03 18:55:30 -0800 2017,Fri Feb 03 00:00:00 -0800 2017,Fri Feb 03 00:00:00 -0800 2017,3,4,"['plot', 'regurgitation', 'couple', 'call', 'f...",plot regurgitation couple call fantasy fiction...
2,202b5764361fb781e18d2e3edeffd599,10852699,840a4fb63336a3fc2213e5016cafbca2,4 stars - Contemporary Romance,Thu Nov 04 10:21:44 -0700 2010,Sat Nov 26 05:48:55 -0800 2011,Fri Nov 25 00:00:00 -0800 2011,Fri Nov 25 00:00:00 -0800 2011,0,0,"['star', 'contemporary', 'romance']",star contemporary romance
3,3519beb3f46f463283ccf9bdbe1622df,11588,6c5865dc54856d1dcb317c3dd42215a8,Another King I've read so many years ago. Back...,Thu Oct 04 03:43:05 -0700 2012,Wed Oct 02 07:17:30 -0700 2013,Thu Jan 01 00:00:00 -0800 1998,NaN,3,4,"['another', 'king', 'read', 'many', 'year', 'a...",another king read many year ago back day read ...
4,4e637c380d40de359012be29091be3b4,15839984,0fdf6c933c0b299ebf5053b8d9a2f950,"so good, SO GOOD. it took a while to suck me i...",Wed May 13 17:39:37 -0700 2015,Sun Jan 29 13:26:50 -0800 2017,Mon Aug 17 00:00:00 -0700 2015,Sun Aug 16 00:00:00 -0700 2015,1,11,"['good', 'good', 'took', 'suck', 'convince', '...",good good took suck convince beginning resist ...


In [31]:
test_copy = test.copy()
columns_not_to_drop = 'destr_review'
for col in test.columns:
    if col not in columns_not_to_drop:
        test.drop(columns=col, inplace=True)

In [32]:
test.head()

,review,destr_review
0,"['cute', 'sweet', 'story', 'girl', 'travel', '...",cute sweet story girl travel neverland fall ca...
1,"['plot', 'regurgitation', 'couple', 'call', 'f...",plot regurgitation couple call fantasy fiction...
2,"['star', 'contemporary', 'romance']",star contemporary romance
3,"['another', 'king', 'read', 'many', 'year', 'a...",another king read many year ago back day read ...
4,"['good', 'good', 'took', 'suck', 'convince', '...",good good took suck convince beginning resist ...


In [36]:
pred = grid_search.best_estimator_.predict(test_vec)

In [37]:
pred

array([4, 4, 4, ..., 4, 5, 4], dtype=int64)

In [40]:
submission = pd.DataFrame({'review_id':test_copy['review_id'],'rating':pred}) 

In [41]:
submission.to_csv('output.csv', index=False)